In [ ]:
import numpy as np
import pandas as pd
import json
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import precision_recall_curve, auc
from matplotlib.pyplot import figure, text
import networkx as nx

import json

import sys
sys.path.append('../')  
import utils

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score
import numpy as np
import pandas as pd

folders = ["s1", "s2", "s3"]
data_path = "../DATA/"
norm = "normPower2"  

results = []

for c in range(len(folders)):
    path = f"../E2_Standard-GCN/dtw_matrices/{folders[c]}/tr_AMR_{norm}_sparse.csv"
    dtw_matrix = pd.read_csv(path)

    X_train = np.load(f"../DATA/{folders[c]}/X_train_tensor_{norm}.npy")
    X_test = np.load(f"../DATA/{folders[c]}/X_test_tensor_{norm}.npy")
    y_train = pd.read_csv(f"../DATA/{folders[c]}/y_train_tensor_{norm}.csv")[['individualMRGerm_stac']].values.ravel() 
    y_test = pd.read_csv(f"../DATA/{folders[c]}/y_test_tensor_{norm}.csv")[['individualMRGerm_stac']].values.ravel()

    # Normalize DTW Matrix and Apply Transformation
    A = dtw_matrix
    I = np.eye(A.shape[0])
    A = A - I 
    A = A / np.linalg.norm(A)  

    X_train_transformed = np.einsum('ij,ptj->pti', A, X_train)
    X_test_transformed = np.einsum('ij,ptj->pti', A, X_test)

    X_train_transformed = X_train_transformed.reshape(X_train_transformed.shape[0], -1)
    X_test_transformed = X_test_transformed.reshape(X_test_transformed.shape[0], -1)

    # Train Logistic Regression
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train_transformed, y_train)

    # Predictions
    y_pred = clf.predict(X_test_transformed)
    y_prob = clf.predict_proba(X_test_transformed)[:, 1] 

    # Metrics Calculation
    test_score = clf.score(X_test_transformed, y_test)  # Accuracy
    roc_auc = roc_auc_score(y_test, y_prob)  # ROC-AUC
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn)  # Sensitivity (Recall)
    specificity = tn / (tn + fp)  # Specificity
    f1 = f1_score(y_test, y_pred)  # F1-score

    # Store Results
    results.append({
        "Split": folders[c],
        "Accuracy": test_score,
        "ROC-AUC": roc_auc,
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "F1-Score": f1
    })

# Convert results to DataFrame
df_results = pd.DataFrame(results)